In [ ]:
# Importazione delle librerie necessarie
import boto3
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sagemaker import get_execution_role
import pickle
import json
import datetime
from time import gmtime, strftime


In [ ]:
# Configurazione della sessione AWS
my_session = boto3.session.Session()
aws_region = my_session.region_name

# Client SageMaker
sagemaker_client = boto3.client('sagemaker', region_name=aws_region)
sagemaker_role = get_execution_role()

# Definizione di alcuni parametri
bucket_name = 'il_tuo_bucket'  # Sostituisci con il nome del tuo bucket S3
s3_prefix = 'path/alla/cartella/train'  # Sostituisci con il percorso della cartella 'train' su S3
data_dir = 'data'


In [ ]:
# Funzione per scaricare i dati da S3
def download_data_from_s3(bucket_name, s3_prefix, local_dir):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    for obj in bucket.objects.filter(Prefix=s3_prefix):
        if not os.path.exists(os.path.dirname(os.path.join(local_dir, obj.key))):
            os.makedirs(os.path.dirname(os.path.join(local_dir, obj.key)), exist_ok=True)
        if not obj.key.endswith('/'):
            bucket.download_file(obj.key, os.path.join(local_dir, obj.key))

# Scarica i dati
download_data_from_s3(bucket_name, s3_prefix, data_dir)


In [ ]:
# Imposta i generatori di dati
train_dir = os.path.join(data_dir, s3_prefix)
batch_size = 32
image_size = (224, 224)

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Salva la mappatura delle classi
class_indices = train_generator.class_indices
with open('class_indices.pkl', 'wb') as f:
    pickle.dump(class_indices, f)


In [ ]:
# Costruisci il modello (puoi personalizzarlo o utilizzare un modello pre-addestrato)
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(train_generator.num_classes, activation='softmax'))

# Compila il modello
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Allena il modello
epochs = 10  # Puoi aumentare il numero di epoche se necessario
model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

# Salva il modello
model.save('card_model.h5')


In [ ]:
# Copia i file necessari nella directory corretta per il Docker build
!mkdir -p model
!cp card_model.h5 model/
!cp class_indices.pkl model/


In [ ]:
%%sh
# Definisci il nome dell'immagine
image_name=card-classification-inference
account=$(aws sts get-caller-identity --query Account --output text)
region=$(aws configure get region)
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:latest"

# Crea il repository ECR se non esiste
aws ecr describe-repositories --repository-names "${image_name}" || aws ecr create-repository --repository-name "${image_name}"

# Effettua il login a ECR
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${account}.dkr.ecr.${region}.amazonaws.com

# Costruisci l'immagine Docker
docker build -t ${image_name} -f Dockerfile.inference .

# Tagga l'immagine
docker tag ${image_name}:latest ${fullname}

# Push dell'immagine su ECR
docker push ${fullname}


In [ ]:
# Definisci il nome del modello e dell'endpoint
model_name = 'card-classification-model'
endpoint_config_name = 'card-classification-endpoint-config'
endpoint_name = 'card-classification-endpoint'

# Crea il modello
create_model_response = sagemaker_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=sagemaker_role,
    PrimaryContainer={
        'Image': fullname,
        # I file del modello sono inclusi nell'immagine Docker, quindi 'ModelDataUrl' non è necessario
    }
)

# Crea la configurazione dell'endpoint
endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "ModelName": model_name,
            "InstanceType": "ml.m5.large",
            "InitialInstanceCount": 1
        }
    ]
)

# Crea l'endpoint
create_endpoint_response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

# Attendi che l'endpoint sia in stato 'InService'
import time

describe_endpoint_response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
status = describe_endpoint_response['EndpointStatus']
print(f"EndpointStatus = {status}")

while status != 'InService':
    time.sleep(30)
    describe_endpoint_response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    status = describe_endpoint_response['EndpointStatus']
    print(f"EndpointStatus = {status}")


In [ ]:
import boto3

# Creazione del client runtime di SageMaker
runtime_client = boto3.client('sagemaker-runtime')

# Funzione per inviare una richiesta all'endpoint
def invoke_endpoint(image_path):
    with open(image_path, 'rb') as f:
        payload = f.read()
    
    response = runtime_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/octet-stream',
        Body=payload
    )
    
    result = json.loads(response['Body'].read().decode())
    print(result)

# Esempio di utilizzo
invoke_endpoint('path/to/your/card_image.jpg')  # Sostituisci con il percorso dell'immagine della carta


In [ ]:
import boto3

# Creazione del client runtime di SageMaker
runtime_client = boto3.client('sagemaker-runtime')

# Funzione per inviare una richiesta all'endpoint
def invoke_endpoint(image_path):
    with open(image_path, 'rb') as f:
        payload = f.read()
    
    response = runtime_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/octet-stream',
        Body=payload
    )
    
    result = json.loads(response['Body'].read().decode())
    print(result)

# Esempio di utilizzo
invoke_endpoint('path/to/your/card_image.jpg')  # Sostituisci con il percorso dell'immagine della carta
